# Python Environment To Demonstrate DW & ETL on MSSQL

#### Important Note: Kindly run the cells one by one. Some cells give output, some don't.

In [1]:
### SET UP THE PYTHON QUERYING ENVIRONMENT ###

## Run this cell only once. Expect no output ##

!pip install pyodbc
import pyodbc
import pandas as pd
import warnings as wn
wn.filterwarnings('ignore')
conn = pyodbc.connect(r'Driver=SQL Server Native Client 11.0;Server=localhost;Database=water_quality;Trusted_Connection=yes;')
cursor = conn.cursor()

# By running this cell, connection is made from this Python environment to a SQL Server 2019 database already on the local machine.

### Extract Phase

In [62]:
### COMBINE ALL 17 RAW IMPORTED TABLES INTO ONE BIG STAGING TABLE ###

## Run this cell only once. Expect no output ##

cursor.execute("""

USE water_quality;

DROP TABLE IF EXISTS dw_water_quality;

SELECT * INTO dw_water_quality FROM _2000;

INSERT INTO dw_water_quality
    SELECT * FROM _2001
        UNION
    SELECT * FROM _2002
        UNION
    SELECT * FROM _2003
        UNION
    SELECT * FROM _2004
        UNION
    SELECT * FROM _2005
        UNION
    SELECT * FROM _2006
        UNION
    SELECT * FROM _2007
        UNION
    SELECT * FROM _2008
        UNION
    SELECT * FROM _2009
        UNION
    SELECT * FROM _2010
        UNION
    SELECT * FROM _2011
        UNION
    SELECT * FROM _2012
        UNION
    SELECT * FROM _2013
        UNION
    SELECT * FROM _2014
        UNION
    SELECT * FROM _2015
        UNION
    SELECT * FROM _2016;
        
            """)


conn.commit()

In [3]:
### EXAMINE THE STAGING TABLE THAT HAS BEEN PREPARED ###

## You can run this cell more than once, but don't re-run this cell after the TRANSFORM PHASE. Expect a dataframe ##

query = """

SELECT * FROM dw_water_quality;

        """

df = pd.read_sql_query(query, conn)
df

# 2348 records. 19 fields.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing
0,1,1.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04T14:10:00,Ammonia(N),Ammoniacal Nitrogen as N,111.0,None,10.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
1,2,9.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-11-15T14:50:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,14.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
2,3,17.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011489,RED HOUSE NURSING HOME SYRESHAM,2000-07-17T10:15:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,65.3000,None,mg/l,FINAL SEWAGE EFFLUENT,False,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,462060.0,240890.0
3,4,25.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Iron - as Fe,Iron,6051.0,None,160.0000,None,ug/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
4,5,33.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,4.0000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343,255,1352.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-05-21T11:31:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,3.7800,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
2344,255,2541.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-43100100,REDE AT COTTONSHOPEFOOT,2015-11-30T10:59:00,Ca Filtered,"Calcium, Dissolved",239.0,None,10.2000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,377820.0,601280.0
2345,256,1353.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,8.6700,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
2346,257,1354.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Nitrate-N,Nitrate as N,117.0,None,6.0400,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0


### Transform Phase

##### Examine the ID1 column

In [4]:
### CHECK THE ID1 COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT ID1 
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 258 distinct values! This column will not be needed in the load phase.

,ID1
0,23
1,238
2,46
3,215
4,69
...,...
253,51
254,151
255,194
256,200


In [5]:
### CHECK THE ID1 COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE ID1 IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values. This column will still not be needed in the load phase.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the ID column

In [7]:
### CHECK THE ID COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT ID 
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 1048 distinct values! This column will not be needed in the load phase.

,ID
0,713.0
1,1981.0
2,1211.0
3,1384.0
4,953.0
...,...
1043,1155.0
1044,791.0
1045,473.0
1046,39.0


In [8]:
### CHECK THE ID COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE ID IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values. This column will still not be needed in the load phase.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the @id column

In [11]:
### CHECK THE @id COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT [@id] 
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 2348 distinct values. This column could serve as a unique key column.

,@id
0,http://environment.data.gov.uk/water-quality/d...
1,http://environment.data.gov.uk/water-quality/d...
2,http://environment.data.gov.uk/water-quality/d...
3,http://environment.data.gov.uk/water-quality/d...
4,http://environment.data.gov.uk/water-quality/d...
...,...
2343,http://environment.data.gov.uk/water-quality/d...
2344,http://environment.data.gov.uk/water-quality/d...
2345,http://environment.data.gov.uk/water-quality/d...
2346,http://environment.data.gov.uk/water-quality/d...


In [12]:
### CHECK THE @id COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE [@id] IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesamplingPoint column

In [13]:
### CHECK THE ID_ COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesamplingPoint 
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 81 distinct values.

,samplesamplingPoint
0,http://environment.data.gov.uk/water-quality/i...
1,http://environment.data.gov.uk/water-quality/i...
2,http://environment.data.gov.uk/water-quality/i...
3,http://environment.data.gov.uk/water-quality/i...
4,http://environment.data.gov.uk/water-quality/i...
...,...
76,http://environment.data.gov.uk/water-quality/i...
77,http://environment.data.gov.uk/water-quality/i...
78,http://environment.data.gov.uk/water-quality/i...
79,http://environment.data.gov.uk/water-quality/i...


In [14]:
### CHECK THE samplesamplingPoint COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesamplingPoint IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesamplingPointnotation column

In [15]:
### CHECK THE samplesamplingPointnotation COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesamplingPointnotation 
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 81 distinct values.

,samplesamplingPointnotation
0,AN-04959
1,AN-04M15
2,MD-36791360
3,AN-02M25
4,AN-01558
...,...
76,AN-04M13
77,AN-02M10
78,MD-03332190
79,AN-011638


In [16]:
### CHECK THE samplesamplingPointnotation COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesamplingPointnotation IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesamplingPointlabel column

In [17]:
### CHECK THE samplesamplingPointlabel COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesamplingPointlabel
    FROM dw_water_quality;
    
    """

df = pd.read_sql_query(query, conn)
df

# 81 distinct locations.

,samplesamplingPointlabel
0,R.TOVE A508 RD.BR.STOKE PARK
1,TRIB.OUZEL BK.D/S BLUEWATER
2,HELMDON BK.HELMDON RD.BR.
3,PRESTON BROOK WINDCROSS
4,CLUBHOUSE GT.LINFORD LAKES
...,...
76,STEANE PARK THE MANOR HOUSE STW
77,FALDINGWORTH
78,BLAKESLEY BK.TRIB.TOVE KINGTHORN MILL
79,SANDHOUSE BACKWASH


In [18]:
### CHECK THE samplesamplingPointlabel COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesamplingPointlabel IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null values.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesampleDateTime column

In [19]:
### CHECK THE samplesampleDateTime COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesampleDateTime
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 1077 distinct timestamps.

,samplesampleDateTime
0,2000-01-04T14:10:00
1,2000-01-10T13:25:00
2,2000-01-24T09:40:00
3,2000-01-24T10:00:00
4,2000-01-24T10:25:00
...,...
1072,2016-12-05T13:58:00
1073,2016-12-05T14:23:00
1074,2016-12-05T15:05:00
1075,2016-12-07T13:00:00


In [20]:
### CHECK THE samplesampleDateTime COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesampleDateTime IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the determinandlabel AND determinanddefinition AND determinandnotation columns

In [21]:
### CHECK THE determinandlabel COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT determinandlabel
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 173 distinct 'sensor' types.

,determinandlabel
0,"1,1,1-TCA"
1,"1,2,4-TCB"
2,1122TetClEth
3,12-DCA
4,"2,3,5,6-Tetr"
...,...
168,WethPresPrec
169,WethPresTemp
170,Weth-Visibty
171,Zinc - as Zn


In [22]:
### CHECK THE determinanddefinition COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT determinanddefinition
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 172 distinct values.

,determinanddefinition
0,"1,1,1-Trichloroethane"
1,"1,1,2,2-Tetrachloroethane"
2,"1,2,4-Trichlorobenzene"
3,"1,2-Dichloroethane"
4,"1,2-Dimethylbenzene :- {o-Xylene}"
...,...
167,Weather : Precipitation
168,Weather : Temperature
169,Weather : Visibility
170,Zinc


In [23]:
### CHECK THE determinandnotation COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT determinandnotation
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 173 distinct values.

,determinandnotation
0,3878.0
1,3410.0
2,6569.0
3,81.0
4,6841.0
...,...
168,9447.0
169,3781.0
170,746.0
171,6619.0


In [24]:
### CHECK THE determinandlabel AND determinanddefinition AND determinandnotation COLUMNS FOR NULL VALUES

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE determinandlabel IS NULL
    OR determinanddefinition IS NULL
    OR determinandnotation IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


In [25]:
### CHECK THE determinandlabel COLUMN FOR OUTLIERS ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT determinandlabel, COUNT(determinandlabel) cnt
    FROM dw_water_quality
GROUP BY determinandlabel 
ORDER BY cnt;

        """

df = pd.read_sql_query(query, conn)
df

# Up to 95 'sensors' out of 163 appear less than three times. Click to the left of the dataframe to scroll.
# This observation will be ignored for this project.

,determinandlabel,cnt
0,"1,1,1-TCA",1
1,"1,2,4-TCB",1
2,1122TetClEth,1
3,"2,3,5,6-Tetr",1
4,"2,3-Xylenol",1
...,...,...
168,N Oxidised,118
169,Orthophospht,126
170,O Diss %sat,131
171,pH,134


##### Examine the resultQualifiernotation column

In [26]:
### CHECK THE resultQualifiernotation COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT resultQualifiernotation 
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 2 distinct values. Null and "<". This column will not be used in the load phase since it contains null values.

,resultQualifiernotation
0,None
1,<


In [27]:
### CHECK THE resultQualifiernotation COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE resultQualifiernotation IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# 1845 null values! This column will not be used in the load phase since it contains a lot of null values.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing
0,1,1.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04T14:10:00,Ammonia(N),Ammoniacal Nitrogen as N,111.0,None,10.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
1,2,9.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-11-15T14:50:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,14.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
2,3,17.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011489,RED HOUSE NURSING HOME SYRESHAM,2000-07-17T10:15:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,65.3000,None,mg/l,FINAL SEWAGE EFFLUENT,False,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,462060.0,240890.0
3,4,25.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Iron - as Fe,Iron,6051.0,None,160.0000,None,ug/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
4,5,33.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,4.0000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,255,1352.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-05-21T11:31:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,3.7800,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
1841,255,2541.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NE-43100100,REDE AT COTTONSHOPEFOOT,2015-11-30T10:59:00,Ca Filtered,"Calcium, Dissolved",239.0,None,10.2000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,377820.0,601280.0
1842,256,1353.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,8.6700,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
1843,257,1354.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Nitrate-N,Nitrate as N,117.0,None,6.0400,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0


##### Examine the result column

In [28]:
### CHECK THE result COLUMN FOR NULL VALUES

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE determinandlabel IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the codedResultInterpretationinterpretation column

In [29]:
### CHECK THE codedResultInterpretationinterpretation COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT codedResultInterpretationinterpretation 
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# This column will not be used in the load phase because it contains only nulls.

,codedResultInterpretationinterpretation
0,None


##### Examine the determinandunitlabel column

In [30]:
### CHECK THE determinandunitlabel COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT determinandunitlabel
    FROM dw_water_quality;
    
        """

df = pd.read_sql_query(query, conn)
df

# 12 unique units.

,determinandunitlabel
0,ntu
1,unitless
2,pres/nf
3,text
4,us/cm
5,ftu
6,%
7,coded
8,mg/l
9,cel


In [31]:
### CHECK THE determinandunitlabel COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE determinandunitlabel IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


In [32]:
### EXAMINE THE determinandunitlabel COLUMN IN THE STAGING TABLE ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT determinandunitlabel, COUNT(determinandunitlabel) cnt
    FROM dw_water_quality
GROUP BY determinandunitlabel
ORDER BY cnt;

        """

df = pd.read_sql_query(query, conn)
df

# 5 out of 12 units appear in less than 50 records.

,determinandunitlabel,cnt
0,unitless,2
1,text,2
2,ntu,16
3,ftu,31
4,coded,45
5,pres/nf,56
6,us/cm,82
7,cel,116
8,phunits,134
9,%,139


In [33]:
### CHECK THE determinandunitlabel COLUMN FOR TOTAL OUTLIERS ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE determinandunitlabel = 'unitless'
        UNION
SELECT * FROM dw_water_quality
    WHERE determinandunitlabel = 'text'
        UNION
SELECT * FROM dw_water_quality
    WHERE determinandunitlabel = 'coded';
    
        """

df = pd.read_sql_query(query, conn)
df

# These 49 records are to be deleted because they seem not to be important considering 
# their corresponding values in the determinanddefinition column.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing
0,1,1.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2001-01-08T12:00:00,NO FLOW/SAMP,No flow /No sample,7668.0,None,1.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
1,1,1.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2005-01-10T10:55:00,NO FLOW/SAMP,No flow /No sample,7668.0,None,0.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
2,1,1.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011262,STEANE PARK THE MANOR HOUSE STW,2013-08-01T11:30:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444.0,None,1.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510.0,239070.0
3,2,11.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01558,ROBIN HOOD PH BUFFLERS HOLT,2001-04-11T11:20:00,NO FLOW/SAMP,No flow /No sample,7668.0,None,0.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),466660.0,235220.0
4,4,4.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011265,BLACKPIT FARM STW STOWE,2013-07-30T12:55:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444.0,None,1.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467660.0,239950.0
5,5,110.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011489,RED HOUSE NURSING HOME SYRESHAM,2011-02-08T13:49:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444.0,None,0.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),462060.0,240890.0
6,10,10.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011489,RED HOUSE NURSING HOME SYRESHAM,2010-02-17T13:50:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444.0,None,1.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),462060.0,240890.0
7,10,10.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2013-06-15T10:06:00,NO FLOW/SAMP,No flow /No sample,7668.0,None,0.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
8,10,82.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2014-10-09T16:44:00,NO FLOW/SAMP,No flow /No sample,7668.0,None,0.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
9,11,11.0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,AN-01558,ROBIN HOOD PH BUFFLERS HOLT,2010-07-20T09:50:00,Site Inspect,Site Inspection : Pass/Fail (1/0),4883.0,None,1.0,None,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),466660.0,235220.0


In [34]:
### CHECK THE determinanddefinition COLUMN INFO OF determinandunitlabel COLUMN ENTRIES THAT LOOKS ODD ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT s.determinanddefinition FROM
    (
    SELECT * FROM dw_water_quality
        WHERE determinandunitlabel = 'unitless'
            UNION
    SELECT * FROM dw_water_quality
        WHERE determinandunitlabel = 'text'	
            UNION
    SELECT * FROM dw_water_quality
        WHERE determinandunitlabel = 'coded'
    ) s;
    
        """

df = pd.read_sql_query(query, conn)
df

# The result summary below shows the reason why these 49 records seem not to be important.

,determinanddefinition
0,Descriptive Site Inspection : Pass/Fail 1/0
1,GCMS Scan : Semi-Volatile Screen
2,GCMS Screen : Target Based multi-residue scree...
3,GCMS Screen : Volatile Screen : Semi Quantitative
4,ICP-MS : Metals screen : Semi-quantitative
5,No flow /No sample
6,Site Inspection : Pass/Fail (1/0)
7,Weather : Precipitation
8,Weather : Temperature
9,Weather : Visibility


##### Examine the sampleisComplianceSample  column

In [35]:
### CHECK THE sampleisComplianceSample COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT sampleisComplianceSample
    FROM dw_water_quality;
        
        """

df = pd.read_sql_query(query, conn)
df

# 2 unique results (1 and 0. This implies "compliant" and "non-compliant" respectively).

,sampleisComplianceSample
0,False
1,True


In [36]:
### CHECK THE sampleisComplianceSample COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE sampleisComplianceSample IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesampledMaterialTypelabel column

In [37]:
### CHECK THE samplesampledMaterialTypelabel COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesampledMaterialTypelabel 
    FROM dw_water_quality;
        
        """

df = pd.read_sql_query(query, conn)
df

# 9 unique values.

,samplesampledMaterialTypelabel
0,ANY TRADE EFFLUENT
1,GROUNDWATER - PURGED/PUMPED/REFILLED
2,RUNNING SURFACE WATER SEDIMENT - <63UM FRACTION
3,FINAL SEWAGE EFFLUENT
4,UNCODED
5,GROUNDWATER
6,ANY SEWAGE
7,ANY WATER
8,RIVER / RUNNING SURFACE WATER


In [38]:
### CHECK THE samplesampledMaterialTypelabel COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesampledMaterialTypelabel IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplepurposelabel column

In [39]:
### CHECK THE samplepurposelabel COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplepurposelabel 
    FROM dw_water_quality;
        
        """

df = pd.read_sql_query(query, conn)
df

# 14 unique values.

,samplepurposelabel
0,UNPLANNED REACTIVE MONITORING FORMAL (POLLUTIO...
1,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY)
2,PLANNED INVESTIGATION (LOCAL MONITORING)
3,MONITORING (UK GOVT POLICY - NOT GQA OR RE)
4,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...
5,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...
6,COMPLIANCE AUDIT (PERMIT)
7,WATER QUALITY UWWTD MONITORING DATA
8,ENVIRONMENTAL MONITORING (GQA & RE ONLY)
9,WATER QUALITY OPERATOR SELF MONITORING COMPLIA...


In [40]:
### CHECK THE samplepurposelabel COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplepurposelabel IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesamplingPointeasting column

In [41]:
### CHECK THE samplesamplingPointeasting COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesamplingPointeasting 
    FROM dw_water_quality;
        
        """

df = pd.read_sql_query(query, conn)
df

# 80 unique values!!

,samplesamplingPointeasting
0,475760.0
1,465800.0
2,479484.0
3,471467.0
4,450491.0
...,...
75,478766.0
76,466660.0
77,494702.0
78,492020.0


In [42]:
### CHECK THE samplesamplingPointeasting COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesamplingPointeasting IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Examine the samplesamplingPointnorthing column

In [43]:
### CHECK THE samplesamplingPointnorthing COLUMN FOR DISTINCT VALUES ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT DISTINCT samplesamplingPointnorthing 
    FROM dw_water_quality;
        
        """

df = pd.read_sql_query(query, conn)
df

# 81 unique results.

,samplesamplingPointnorthing
0,232851.0
1,237506.0
2,228850.0
3,592620.0
4,230220.0
...,...
76,238550.0
77,250870.0
78,240255.0
79,228700.0


In [44]:
### CHECK THE samplesamplingPointnorthing COLUMN FOR NULL VALUES ###

## You can run this cell more than once. Expect an empty dataframe ##

query = """

SELECT * FROM dw_water_quality
    WHERE samplesamplingPointnorthing IS NULL;
    
        """

df = pd.read_sql_query(query, conn)
df

# No null records.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing


##### Data cleaning...

In [63]:
### DATA CLEANING 1: TO REMOVE UNNECESSARY RECORDS ###

## Deleting is done using the distinct column "@id" as a unique key column!

## Run this cell only once. Expect no output ##

cursor.execute("""

DELETE FROM dw_water_quality 
    WHERE [@id] IN
        (
        SELECT s.[@id] FROM
            (
            SELECT * FROM dw_water_quality
                WHERE determinandunitlabel = 'unitless'
                    UNION
            SELECT * FROM dw_water_quality
                WHERE determinandunitlabel = 'text'	
                    UNION
            SELECT * FROM dw_water_quality
                WHERE determinandunitlabel = 'coded'
            ) s
        );
        
            """)

conn.commit()

In [64]:
### DATA CLEANING 2: TO REMOVE REDUNDANT PARTS OF ENTRIES ###

## To strip off all appearances of “http://environment.data.gov.uk/water-quality” from the @id and samplesamplingpoint columns

## Run this cell only once. Expect no output ##

cursor.execute("""

UPDATE dw_water_quality
    SET [@id] = REPLACE([@id], 'http://environment.data.gov.uk/water-quality', '');

            """)

conn.commit()



cursor.execute("""

UPDATE dw_water_quality
    SET samplesamplingpoint = REPLACE(samplesamplingpoint, 'http://environment.data.gov.uk/water-quality', '');
    
            """)

conn.commit()

In [48]:
### EXAMINE THE RESIDUAL STAGING TABLE SO FAR ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dw_water_quality;

        """

df = pd.read_sql_query(query, conn)
df

# 49 out of 2348 records were deleted. 2299 records remaining.

,ID1,ID,@id,samplesamplingPoint,samplesamplingPointnotation,samplesamplingPointlabel,samplesampleDateTime,determinandlabel,determinanddefinition,determinandnotation,resultQualifiernotation,result,codedResultInterpretationinterpretation,determinandunitlabel,samplesampledMaterialTypelabel,sampleisComplianceSample,samplepurposelabel,samplesamplingPointeasting,samplesamplingPointnorthing
0,1,1.0,/data/measurement/AN-687353-0111,/id/sampling-point/AN-011396,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04T14:10:00,Ammonia(N),Ammoniacal Nitrogen as N,111.0,None,10.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
1,2,9.0,/data/measurement/AN-1053332-0135,/id/sampling-point/AN-011396,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-11-15T14:50:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,14.4000,None,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020.0,233400.0
2,3,17.0,/data/measurement/AN-1037010-0135,/id/sampling-point/AN-011489,AN-011489,RED HOUSE NURSING HOME SYRESHAM,2000-07-17T10:15:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,65.3000,None,mg/l,FINAL SEWAGE EFFLUENT,False,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...,462060.0,240890.0
3,4,25.0,/data/measurement/AN-689676-6051,/id/sampling-point/AN-01M04,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Iron - as Fe,Iron,6051.0,None,160.0000,None,ug/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
4,5,33.0,/data/measurement/AN-689676-0135,/id/sampling-point/AN-01M04,AN-01M04,R.OUSE A422 RD.BR.BRACKLEY,2000-01-24T10:55:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,4.0000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,459427.0,236819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,255,1352.0,/data/measurement/AN-1673460-0135,/id/sampling-point/AN-02M10,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-05-21T11:31:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,3.7800,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
2295,255,2541.0,/data/measurement/NE-1542543-0239,/id/sampling-point/NE-43100100,NE-43100100,REDE AT COTTONSHOPEFOOT,2015-11-30T10:59:00,Ca Filtered,"Calcium, Dissolved",239.0,None,10.2000,None,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,377820.0,601280.0
2296,256,1353.0,/data/measurement/AN-1676129-0135,/id/sampling-point/AN-02M10,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Sld Sus@105C,"Solids, Suspended at 105 C",135.0,None,8.6700,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0
2297,257,1354.0,/data/measurement/AN-1676129-0117,/id/sampling-point/AN-02M10,AN-02M10,PADBURY BK.TRIB.OUSE KINGSBRIDGE FORD,2012-06-14T14:29:00,Nitrate-N,Nitrate as N,117.0,None,6.0400,None,mg/l,RIVER / RUNNING SURFACE WATER,False,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY),470377.0,228850.0


##### At this point, column selection, BUS plan, as well as fact and dimension tables design should be done.

### Load Phase

In [65]:
### TO CREATE DIMENSION TABLES AND FACT TABLE AND LOAD THEM ###

## Run this cell only once. Expect no output ##


cursor.execute("""

--- To rename the 15 columns ---

EXEC sp_rename 'dw_water_quality.[@id]', 'measurementMetadata', 'COLUMN';
    
EXEC sp_rename 'dw_water_quality.samplesamplingPoint', 'locationMetadata', 'COLUMN';
    
EXEC sp_rename 'dw_water_quality.samplesamplingPointnotation', 'locationNotation', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplesamplingPointlabel', 'measurementLocation', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplesampleDateTime', 'measurementDateTime', 'COLUMN';

EXEC sp_rename 'dw_water_quality.determinandlabel', 'sensorType', 'COLUMN';

EXEC sp_rename 'dw_water_quality.determinanddefinition', 'sensorTypeDefinition', 'COLUMN';
    
EXEC sp_rename 'dw_water_quality.determinandnotation', 'sensorNotation', 'COLUMN';

EXEC sp_rename 'dw_water_quality.result', 'measurement', 'COLUMN';

EXEC sp_rename 'dw_water_quality.determinandunitlabel', 'sensorUnit', 'COLUMN';
    
EXEC sp_rename 'dw_water_quality.sampleisComplianceSample', 'sampleCompliance', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplesampledMaterialTypelabel', 'samplingMaterial', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplepurposelabel', 'samplingPurpose', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplesamplingPointeasting', 'locationEasting', 'COLUMN';

EXEC sp_rename 'dw_water_quality.samplesamplingPointnorthing', 'locationNorthing', 'COLUMN';



--- To typecast the measurementDateTime column into DATETIME datatype ---

ALTER TABLE dw_water_quality ALTER COLUMN measurementDateTime DATETIME;


--- To add a measurementYear column of INTEGER datatype ---

ALTER TABLE dw_water_quality ADD measurementYear INT;

            """)

conn.commit()



cursor.execute("""

--- To populate the measurementYear column with year data from measurementDateTime column ---

UPDATE dw_water_quality SET measurementYear = YEAR(measurementDateTime) WHERE measurementMetadata = measurementMetadata;

--- or, UPDATE dw_water_quality SET measurementYear = DATENAME(year, measurementDateTime) WHERE measurementMetadata = measurementMetadata;


--- To add a measurementWeek column of INTEGER datatype ---

ALTER TABLE dw_water_quality ADD measurementWeek INT;

            """)

conn.commit()



cursor.execute("""

--- To populate the measurementWeek column with week data from measurementDateTime column ---

UPDATE dw_water_quality SET measurementWeek = DATEPART(WEEK, measurementDateTime) WHERE measurementMetadata = measurementMetadata;


--- To add a measurementMonth column of NVARCHAR datatype ---

ALTER TABLE dw_water_quality ADD measurementMonth NVARCHAR(15);

            """)

conn.commit()



cursor.execute("""

--- To populate the measurementMonth column with week data from measurementDateTime column ---

UPDATE dw_water_quality SET measurementMonth = DATENAME(MONTH, measurementDateTime) WHERE measurementMetadata = measurementMetadata;

            """)

conn.commit()



cursor.execute("""

--- To create 9 dimension tables and a fact table ---

CREATE TABLE dimLocationTable (
    locationID INT NOT NULL PRIMARY KEY IDENTITY,
    locationMetadata NVARCHAR(255) NOT NULL,
    locationNotation NVARCHAR(255) NOT NULL,
    measurementLocation NVARCHAR(255) NOT NULL,
    locationEasting INT NOT NULL,
    locationNorthing INT NOT NULL
    );

CREATE TABLE dimPurposeTable (
    purposeID INT NOT NULL PRIMARY KEY IDENTITY,
    samplingPurpose NVARCHAR(255) NOT NULL
    );
    
CREATE TABLE dimComplianceTable (
    complianceID INT NOT NULL PRIMARY KEY IDENTITY,
    sampleCompliance INT NOT NULL
    );
    
CREATE TABLE dimSampleTable (
    sampleID INT NOT NULL PRIMARY KEY IDENTITY,
    samplingMaterial NVARCHAR(255) NOT NULL,
    );    
    
CREATE TABLE dimSensorTable (
    sensorID INT NOT NULL PRIMARY KEY IDENTITY,
    sensorType NVARCHAR(255) NOT NULL,
    sensorTypeDefinition NVARCHAR(255) NOT NULL,
    sensorNotation INT NOT NULL,
    sensorUnit NVARCHAR(255) NOT NULL
    );

CREATE TABLE dimYearTable (
    yearID INT NOT NULL PRIMARY KEY IDENTITY,
    measurementYear INT NOT NULL
    );

CREATE TABLE dimWeekTable (
    weekID INT NOT NULL PRIMARY KEY IDENTITY,
    measurementWeek INT NOT NULL
    );

CREATE TABLE dimMonthTable (
    monthID INT NOT NULL PRIMARY KEY IDENTITY,
    measurementMonth NVARCHAR(15) NOT NULL
    );

CREATE TABLE dimTimeTable (
    dateTimeID INT NOT NULL PRIMARY KEY IDENTITY,
    measurementDateTime DATETIME NOT NULL,
    yearID INT NOT NULL FOREIGN KEY REFERENCES dimYearTable(yearID),
    weekID INT NOT NULL FOREIGN KEY REFERENCES dimWeekTable(weekID),
    monthID INT NOT NULL FOREIGN KEY REFERENCES dimMonthTable(monthID)
    );

CREATE TABLE factMeasurementsTable (
    factID INT NOT NULL PRIMARY KEY IDENTITY,
    locationID INT NOT NULL FOREIGN KEY REFERENCES dimLocationTable(locationID),
    dateTimeID INT NOT NULL FOREIGN KEY REFERENCES dimTimeTable(dateTimeID),
    sensorID INT NOT NULL FOREIGN KEY REFERENCES dimSensorTable(sensorID),
    purposeID INT NOT NULL FOREIGN KEY REFERENCES dimPurposeTable(purposeID),
    complianceID INT NOT NULL FOREIGN KEY REFERENCES dimComplianceTable(complianceID),
    sampleID INT NOT NULL FOREIGN KEY REFERENCES dimSampleTable(sampleID),
    measurement FLOAT NOT NULL,
    sensorUnit NVARCHAR(255) NOT NULL
    );
  
            """)

conn.commit()



cursor.execute("""

--- To add reference ID columns with INTEGER datatype to the staging table from the 9 dimension tables ---

ALTER TABLE dw_water_quality ADD locationID INT;

ALTER TABLE dw_water_quality ADD dateTimeID INT;

ALTER TABLE dw_water_quality ADD sensorID INT;

ALTER TABLE dw_water_quality ADD yearID INT;

ALTER TABLE dw_water_quality ADD monthID INT;

ALTER TABLE dw_water_quality ADD weekID INT;

ALTER TABLE dw_water_quality ADD purposeID INT;

ALTER TABLE dw_water_quality ADD complianceID INT;

ALTER TABLE dw_water_quality ADD sampleID INT;

            """)

conn.commit()



cursor.execute("""

--- To load each of the 9 dimension tables with data as well as update the staging table with their IDs ---

INSERT INTO dimLocationTable(locationMetadata, locationNotation, measurementLocation, locationEasting, locationNorthing)
    SELECT DISTINCT locationMetadata, locationNotation, measurementLocation, locationEasting, locationNorthing
        FROM dw_water_quality 
    ORDER BY locationMetadata;

UPDATE dw_water_quality 
    SET dw_water_quality.locationID = dimLocationTable.locationID 
        FROM dw_water_quality 
    INNER JOIN dimLocationTable 
        ON dw_water_quality.measurementLocation = dimLocationTable.measurementLocation;

INSERT INTO dimSensorTable(sensorType, sensorTypeDefinition, sensorNotation, sensorUnit)
    SELECT DISTINCT sensorType, sensorTypeDefinition, sensorNotation, sensorUnit
        FROM dw_water_quality 
    ORDER BY sensorType;

UPDATE dw_water_quality 
    SET dw_water_quality.sensorID = dimSensorTable.sensorID 
        FROM dw_water_quality 
    INNER JOIN dimSensorTable 
        ON dw_water_quality.sensorType = dimSensorTable.sensorType;

INSERT INTO dimYearTable(measurementYear)
    SELECT DISTINCT measurementYear 
        FROM dw_water_quality 
    ORDER BY measurementYear;

UPDATE dw_water_quality 
    SET dw_water_quality.yearID = dimYearTable.yearID 
        FROM dw_water_quality 
    INNER JOIN dimYearTable 
        ON dw_water_quality.measurementYear = dimYearTable.measurementYear;

INSERT INTO dimWeekTable(measurementWeek)
    SELECT DISTINCT measurementWeek 
        FROM dw_water_quality 
    ORDER BY measurementWeek;

UPDATE dw_water_quality 
    SET dw_water_quality.weekID = dimWeekTable.weekID 
        FROM dw_water_quality 
    INNER JOIN dimWeekTable 
        ON dw_water_quality.measurementWeek = dimWeekTable.measurementWeek;

INSERT INTO dimMonthTable(measurementMonth)
    SELECT s.measurementMonth FROM 
        (
        SELECT DISTINCT measurementMonth, DATEPART(MONTH, measurementDateTime) c 
            FROM dw_water_quality
        ) s 
    ORDER BY c ASC;

UPDATE dw_water_quality 
    SET dw_water_quality.monthID = dimMonthTable.monthID 
        FROM dw_water_quality 
    INNER JOIN dimMonthTable 
        ON dw_water_quality.measurementMonth = dimMonthTable.measurementMonth;

INSERT INTO dimTimeTable(measurementDateTime, yearID, weekID, monthID)
    SELECT DISTINCT measurementDateTime, yearID, weekID, monthID 
        FROM dw_water_quality 
    ORDER BY yearID ASC;

UPDATE dw_water_quality 
    SET dw_water_quality.dateTimeID = dimTimeTable.dateTimeID 
        FROM dw_water_quality 
    INNER JOIN dimTimeTable 
        ON dw_water_quality.measurementDateTime = dimTimeTable.measurementDateTime
    AND dw_water_quality.yearID = dimTimeTable.yearID
    AND dw_water_quality.monthID = dimTimeTable.monthID
    AND dw_water_quality.weekID = dimTimeTable.weekID;
    
INSERT INTO dimPurposeTable(samplingPurpose)
    SELECT DISTINCT samplingPurpose 
        FROM dw_water_quality 
    ORDER BY samplingPurpose;

UPDATE dw_water_quality
    SET dw_water_quality.purposeID = dimPurposeTable.purposeID 
        FROM dw_water_quality 
    INNER JOIN dimPurposeTable 
        ON dw_water_quality.samplingPurpose = dimPurposeTable.samplingPurpose;
        
INSERT INTO dimComplianceTable(sampleCompliance)
    SELECT DISTINCT sampleCompliance 
        FROM dw_water_quality 
    ORDER BY sampleCompliance;

UPDATE dw_water_quality
    SET dw_water_quality.complianceID = dimComplianceTable.complianceID 
        FROM dw_water_quality 
    INNER JOIN dimComplianceTable 
        ON dw_water_quality.sampleCompliance = dimComplianceTable.sampleCompliance;
        
INSERT INTO dimSampleTable(samplingMaterial)
    SELECT DISTINCT samplingMaterial 
        FROM dw_water_quality 
    ORDER BY samplingMaterial;

UPDATE dw_water_quality
    SET dw_water_quality.sampleID = dimSampleTable.sampleID 
        FROM dw_water_quality 
    INNER JOIN dimSampleTable 
        ON dw_water_quality.samplingMaterial = dimSampleTable.samplingMaterial;
        
INSERT INTO factMeasurementsTable (
    locationID, dateTimeID, sensorID, purposeID, complianceID, sampleID, measurement, sensorUnit
    ) 
    SELECT 
        l.locationID, t.dateTimeID, s.sensorID, p.purposeID, c.complianceID, sm.sampleID, wq.measurement, s.sensorUnit
    FROM dw_water_quality wq
        INNER JOIN dimLocationTable l 
    ON wq.measurementLocation = l.measurementLocation
        INNER JOIN dimSensorTable s 
    ON wq.sensorType = s.sensorType
        INNER JOIN dimPurposeTable p 
    ON wq.samplingPurpose = p.samplingPurpose
        INNER JOIN dimComplianceTable c 
    ON wq.sampleCompliance = c.sampleCompliance
        INNER JOIN dimSampleTable sm 
    ON wq.samplingMaterial = sm.samplingMaterial
        INNER JOIN dimTimeTable t 
    ON wq.measurementDateTime = t.measurementDateTime
        INNER JOIN dimYearTable y 
    ON wq.measurementYear = y.measurementYear
        INNER JOIN dimWeekTable w 
    ON wq.measurementWeek = w.measurementWeek
        INNER JOIN dimMonthTable m 
    ON wq.measurementMonth = m.measurementMonth;

            """)

conn.commit()

###### At this point the ER (entity relationship) diagram or schema diagram should be ready to view in SQL Server Management Studio.

##### The journey so far: The 9 dimension tables (including the Extended Time table), and the fact table

In [66]:
### THE factMeasurementsTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM factMeasurementsTable;

        """

df = pd.read_sql_query(query, conn)
df

# 2299 ENTRIES

,factID,locationID,dateTimeID,sensorID,purposeID,complianceID,sampleID,measurement,sensorUnit
0,1,3,1,24,1,2,4,10.4000,mg/l
1,2,3,52,144,1,2,4,14.4000,mg/l
2,3,4,34,144,10,1,4,65.3000,mg/l
3,4,10,6,88,4,1,7,160.0000,ug/l
4,5,10,6,144,4,1,7,4.0000,mg/l
...,...,...,...,...,...,...,...,...,...
2294,2295,26,668,144,8,1,7,3.7800,mg/l
2295,2296,69,949,43,4,1,7,10.2000,mg/l
2296,2297,26,670,144,8,1,7,8.6700,mg/l
2297,2298,26,670,111,8,1,7,6.0400,mg/l


In [67]:
### THE dimLocationTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimLocationTable;

        """

df = pd.read_sql_query(query, conn)
df

# 72 ENTRIES

,locationID,locationMetadata,locationNotation,measurementLocation,locationEasting,locationNorthing
0,1,/id/sampling-point/AN-011262,AN-011262,STEANE PARK THE MANOR HOUSE STW,455510,239070
1,2,/id/sampling-point/AN-011265,AN-011265,BLACKPIT FARM STW STOWE,467660,239950
2,3,/id/sampling-point/AN-011396,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,467020,233400
3,4,/id/sampling-point/AN-011489,AN-011489,RED HOUSE NURSING HOME SYRESHAM,462060,240890
4,5,/id/sampling-point/AN-011553,AN-011553,"BP CONNECT, BRACKLEY, NORTHAMPTONSHIRE",459250,238550
...,...,...,...,...,...,...
67,68,/id/sampling-point/NE-43100098,NE-43100098,CHIRDON BURN AT TARSET,378320,585035
68,69,/id/sampling-point/NE-43100100,NE-43100100,REDE AT COTTONSHOPEFOOT,377820,601280
69,70,/id/sampling-point/NE-45401467,NE-45401467,BISHOPTON BECK D/S MILL BR PUMPING STN,437930,522500
70,71,/id/sampling-point/NE-45401488,NE-45401488,MARTON WEST BECK BELOW LYTTON STREET,450491,519949


In [68]:
### THE dimSensorTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimSensorTable;

        """

df = pd.read_sql_query(query, conn)
df

# 163 ENTRIES

,sensorID,sensorType,sensorTypeDefinition,sensorNotation,sensorUnit
0,1,"1,1,1-TCA","1,1,1-Trichloroethane",3268,ug/l
1,2,"1,2,4-TCB","1,2,4-Trichlorobenzene",9051,ug/l
2,3,1122TetClEth,"1,1,2,2-Tetrachloroethane",7531,ug/l
3,4,12-DCA,"1,2-Dichloroethane",3272,ug/l
4,5,"2,3,5,6-Tetr","2,3,5,6-Tetrachloroaniline",9350,ug/l
...,...,...,...,...,...
158,159,Turbidity,Turbidity,68,ftu
159,160,TurbidityNTU,Turbidity,6396,ntu
160,161,Uranium - U,Uranium,6473,ug/l
161,162,Zinc - as Zn,Zinc,6455,ug/l


In [70]:
### THE dimPurposeTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimPurposeTable;

        """

df = pd.read_sql_query(query, conn)
df

# 14 ENTRIES

,purposeID,samplingPurpose
0,1,COMPLIANCE AUDIT (PERMIT)
1,2,COMPLIANCE FORMAL (PERMIT)
2,3,ENVIRONMENTAL MONITORING (GQA & RE ONLY)
3,4,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...
4,5,MONITORING (NATIONAL AGENCY POLICY)
5,6,MONITORING (UK GOVT POLICY - NOT GQA OR RE)
6,7,PLANNED INVESTIGATION (LOCAL MONITORING)
7,8,PLANNED INVESTIGATION (NATIONAL AGENCY POLICY)
8,9,STATUTORY FAILURES (FOLLOW UPS AT NON-DESIGNAT...
9,10,UNPLANNED REACTIVE MONITORING (POLLUTION INCID...


In [71]:
### THE dimComplianceTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimComplianceTable;

        """

df = pd.read_sql_query(query, conn)
df

# 2 ENTRIES

,complianceID,sampleCompliance
0,1,0
1,2,1


In [72]:
### THE dimSampleTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimSampleTable;

        """

df = pd.read_sql_query(query, conn)
df

# 9 ENTRIES

,sampleID,samplingMaterial
0,1,ANY SEWAGE
1,2,ANY TRADE EFFLUENT
2,3,ANY WATER
3,4,FINAL SEWAGE EFFLUENT
4,5,GROUNDWATER
5,6,GROUNDWATER - PURGED/PUMPED/REFILLED
6,7,RIVER / RUNNING SURFACE WATER
7,8,RUNNING SURFACE WATER SEDIMENT - <63UM FRACTION
8,9,UNCODED


In [73]:
### THE dimTimeTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimTimeTable;

        """

df = pd.read_sql_query(query, conn)
df

# 1049 ENTRIES

,dateTimeID,measurementDateTime,yearID,weekID,monthID
0,1,2000-01-04 14:10:00,1,2,1
1,2,2000-01-10 13:25:00,1,3,1
2,3,2000-01-24 09:40:00,1,5,1
3,4,2000-01-24 10:00:00,1,5,1
4,5,2000-01-24 10:25:00,1,5,1
...,...,...,...,...,...
1044,1045,2016-12-05 13:58:00,17,50,12
1045,1046,2016-12-05 14:23:00,17,50,12
1046,1047,2016-12-05 15:05:00,17,50,12
1047,1048,2016-12-07 13:00:00,17,50,12


In [74]:
### THE dimYearTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimYearTable;

        """

df = pd.read_sql_query(query, conn)
df

# 17 ENTRIES

,yearID,measurementYear
0,1,2000
1,2,2001
2,3,2002
3,4,2003
4,5,2004
5,6,2005
6,7,2006
7,8,2007
8,9,2008
9,10,2009


In [75]:
### THE dimWeekTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimWeekTable;

        """

df = pd.read_sql_query(query, conn)
df

# 52 ENTRIES

,weekID,measurementWeek
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9
9,10,10


In [76]:
### THE dimMonthTable ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT * FROM dimMonthTable;

        """

df = pd.read_sql_query(query, conn)
df

# 12 ENTRIES

,monthID,measurementMonth
0,1,January
1,2,February
2,3,March
3,4,April
4,5,May
5,6,June
6,7,July
7,8,August
8,9,September
9,10,October


### Querying our Data Warehouse Implementation To Get Information

In [77]:
### The list of water sensors measured by type by month ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT 
    s.measurementMonth, s.sensorType, s.numberOfSensors FROM
    (
    SELECT 
        mt.measurementMonth, 
        st.sensorType, 
        COUNT(measurement) numberOfSensors
    FROM 
        dimSensorTable st 
        INNER JOIN factMeasurementsTable fm ON st.sensorID = fm.sensorID
        INNER JOIN dimTimeTable tt ON fm.dateTimeID = tt.dateTimeID
        INNER JOIN dimMonthTable mt ON tt.monthID = mt.monthID
    GROUP BY 
        st.sensorType, mt.measurementMonth 
    ) s 
JOIN dimMonthTable mt ON s.measurementMonth = mt.measurementMonth
ORDER BY 
    mt.monthID, s.sensorType;
    
        """

df = pd.read_sql_query(query, conn)
df

# 634 rows returned.

,measurementMonth,sensorType,numberOfSensors
0,January,"2,4,6-T",1
1,January,"2,6-Xylenol",1
2,January,"3,4 Xylenol",1
3,January,Alky pH 4.5,4
4,January,Ammonia(N),20
...,...,...,...
629,December,Sodium - Na,1
630,December,Temp Water,10
631,December,Turbidity,4
632,December,TurbidityNTU,2


In [80]:
### The number of sensor measurements collected by type by week ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT 
    wt.measurementWeek, 
    st.sensorType, 
    COUNT(measurement) numberOfMeasurements
FROM 
    dimSensorTable st 
    INNER JOIN factMeasurementsTable fm ON st.sensorID = fm.sensorID
    INNER JOIN dimTimeTable tt ON fm.dateTimeID = tt.dateTimeID
    INNER JOIN dimWeekTable wt ON tt.weekID = wt.weekID
GROUP BY 
    st.sensorType, wt.measurementWeek
ORDER BY
    wt.measurementWeek, st.sensorType;
    
        """

df = pd.read_sql_query(query, conn)
df

# 1258 rows returned.

,measurementWeek,sensorType,numberOfMeasurements
0,1,Metaldehyde,1
1,1,Sld Sus@105C,1
2,2,Alky pH 4.5,1
3,2,Ammonia(N),8
4,2,Cadmium - Cd,1
...,...,...,...
1253,51,pH,1
1254,51,Phosphorus-P,1
1255,51,Turbidity,2
1256,51,Zinc - as Zn,1


In [38]:
### The number of measurements made by location by month ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT 
    s.measurementMonth, s.measurementLocation, s.numberOfMeasurements FROM
    (
    SELECT 
        mt.measurementMonth, 
        lt.measurementLocation, 
        COUNT(measurement) numberOfMeasurements
    FROM 
        dimLocationTable lt 
        INNER JOIN factMeasurementsTable fm ON lt.locationID = fm.locationID
        INNER JOIN dimTimeTable tt ON fm.dateTimeID = tt.dateTimeID
        INNER JOIN dimMonthTable mt ON tt.monthID = mt.monthID
    GROUP BY 
        lt.measurementLocation, mt.measurementMonth
    ) s
JOIN dimMonthTable mt ON s.measurementMonth = mt.measurementMonth
ORDER BY 
    mt.monthID, s.measurementLocation;
    
        """

df = pd.read_sql_query(query, conn)
df

# 399 rows returned.

,measurementMonth,measurementLocation,numberOfMeasurements
0,January,APOLLO OFFICE UNITS RADCLIVE RD GAWCOTT,4
1,January,BISHOPTON BECK D/S MILL BR PUMPING STN,1
2,January,BLAKESLEY BK.TRIB.TOVE KINGTHORN MILL,2
3,January,BUCKINGHAM GOLF CLUB TINGEWICK RD.,9
4,January,CASWELL BK.TRIB.TOVE CASWELL,3
...,...,...,...
394,December,SULGRAVE BK.WESTON RD.BR.,1
395,December,SYRESHAM STR.TRIB.OUSE.A43 RD.BR.KINDSHL,15
396,December,TRIB.CLIPSTONE BK.RD.BR.MANOR FARM,1
397,December,TRIB.OUZEL BK.D/S BLUEWATER,1


In [81]:
### The average number of measurements covered for pH by year ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT 
    yt.measurementYear, 
    COUNT(measurement) numberOfMeasurements
FROM 
    dimSensorTable st 
    INNER JOIN factMeasurementsTable fm ON st.sensorID = fm.sensorID
    INNER JOIN dimTimeTable tt ON fm.dateTimeID = tt.dateTimeID
    INNER JOIN dimYearTable yt ON tt.yearID = yt.yearID
WHERE 
    st.sensorType = 'pH'
GROUP BY 
    yt.measurementYear
ORDER BY 
    yt.measurementYear;
    
        """

df = pd.read_sql_query(query, conn)
df

# 16 rows returned. It is immediately observed that there were no pH measurements recorded for the year 2013.

,measurementYear,numberOfMeasurements
0,2000,5
1,2001,6
2,2002,6
3,2003,11
4,2004,6
5,2005,9
6,2006,4
7,2007,4
8,2008,11
9,2009,6


In [82]:
### The average value of nitrate measurements by locations by year ###

## You can run this cell more than once. Expect a dataframe ##

query = """

SELECT 
    yt.measurementYear,
    lt.measurementLocation,
    ROUND(AVG(measurement), 2) averageValuesOfNitrate
FROM 
    dimLocationTable lt 
    INNER JOIN factMeasurementsTable fm ON lt.locationID = fm.locationID
    INNER JOIN dimSensorTable st ON fm.sensorID = st.sensorID
    INNER JOIN dimTimeTable tt ON fm.dateTimeID = tt.dateTimeID
    INNER JOIN dimYearTable yt ON tt.yearID = yt.yearID
WHERE 
    st.sensorType = 'Nitrate-N'
GROUP BY 
    lt.measurementLocation, yt.measurementYear
ORDER BY 
    yt.measurementYear, lt.measurementLocation;
    
        """

df = pd.read_sql_query(query, conn)
df

# 42 rows returned. It also observed that there were no Nitrate measurements recorded for the years 2001, 2002, and 2013.

,measurementYear,measurementLocation,averageValuesOfNitrate
0,2000,R.OUSE A422 RD.BR.BRACKLEY,6.45
1,2003,R.OUSE A422 RD.BR.BRACKLEY,7.82
2,2003,R.OUSE BOURTON MILL,14.30
3,2004,PRESTON BROOK WINDCROSS,5.68
4,2004,R.OUSE BOURTON MILL,9.76
5,2004,R.OUSE WATER STRATFORD RD.BR.,12.80
6,2005,R.OUSE A422 RD.BR.BRACKLEY,6.61
7,2006,R.OUSE A422 RD.BR.BRACKLEY,5.52
8,2006,R.OUSE BOURTON MILL,12.29
9,2007,R.OUSE A422 RD.BR.BRACKLEY,6.83


In [83]:
### Endeavour to run this cell in order to close the connection to the database.

conn.close()

##### End of demonstration. Thank you for staying with me on this!